In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stockstats import StockDataFrame as Sdf
from ta import *
%matplotlib inline

#stock = pd.read_csv('MSFT_80s_temp3.csv')
stock = pd.read_csv('dow_jones.csv')
#stock = pd.read_csv('JPM_80s.csv')
output = stock

"""

# Converter para dados semanais

stock['Date'] = pd.to_datetime(stock['Date'])
stock.set_index('Date', inplace=True)
stock.sort_index(inplace=True)

def take_first(array_like):
    return array_like[0]

def take_last(array_like):
    return array_like[-1]

output = stock.resample('W',                                 # Weekly resample
                    how={'Open': take_first, 
                         'High': 'max',
                         'Low': 'min',
                         'Close': take_last,
                         'Volume': 'sum'}, 
                    loffset=pd.offsets.timedelta(days=-6))  # to put the labels to Monday

output = stock
output = output[['Open', 'High', 'Low', 'Close', 'Volume']]# Converter para dados semanais

stock['Date'] = pd.to_datetime(stock['Date'])
stock.set_index('Date', inplace=True)
stock.sort_index(inplace=True)

def take_first(array_like):
    return array_like[0]

def take_last(array_like):
    return array_like[-1]

output = stock.resample('W',                                 # Weekly resample
                    how={'Open': take_first, 
                         'High': 'max',
                         'Low': 'min',
                         'Close': take_last,
                         'Volume': 'sum'}, 
                    loffset=pd.offsets.timedelta(days=-6))  # to put the labels to Monday

output = stock
output = output[['Open', 'High', 'Low', 'Close', 'Volume']]
"""

"\n\n# Converter para dados semanais\n\nstock['Date'] = pd.to_datetime(stock['Date'])\nstock.set_index('Date', inplace=True)\nstock.sort_index(inplace=True)\n\ndef take_first(array_like):\n    return array_like[0]\n\ndef take_last(array_like):\n    return array_like[-1]\n\noutput = stock.resample('W',                                 # Weekly resample\n                    how={'Open': take_first, \n                         'High': 'max',\n                         'Low': 'min',\n                         'Close': take_last,\n                         'Volume': 'sum'}, \n                    loffset=pd.offsets.timedelta(days=-6))  # to put the labels to Monday\n\noutput = stock\noutput = output[['Open', 'High', 'Low', 'Close', 'Volume']]# Converter para dados semanais\n\nstock['Date'] = pd.to_datetime(stock['Date'])\nstock.set_index('Date', inplace=True)\nstock.sort_index(inplace=True)\n\ndef take_first(array_like):\n    return array_like[0]\n\ndef take_last(array_like):\n    return array_li

In [31]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89604 entries, 0 to 89603
Data columns (total 7 columns):
Date         89604 non-null object
Open         89604 non-null float64
High         89604 non-null float64
Low          89604 non-null float64
Close        89604 non-null float64
Adj Close    89604 non-null float64
Volume       89604 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 4.8+ MB


In [32]:
output.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-01-02,0.718750,0.721354,0.703125,0.703125,0.018480,3451200
1,1980-01-03,0.703125,0.723958,0.697917,0.721354,0.018959,3960000
2,1980-01-04,0.721354,0.729167,0.718750,0.729167,0.019165,1694400
3,1980-01-07,0.729167,0.731771,0.723958,0.726563,0.019096,4396800
4,1980-01-08,0.726563,0.734375,0.726563,0.731771,0.019233,3244800
5,1980-01-09,0.731771,0.744792,0.729167,0.736979,0.019370,11568000
6,1980-01-10,0.736979,0.744792,0.731771,0.736979,0.019370,19444800
7,1980-01-11,0.736979,0.750000,0.734375,0.739583,0.019438,6240000
8,1980-01-14,0.739583,0.744792,0.729167,0.736979,0.019370,2145600
9,1980-01-15,0.736979,0.736979,0.718750,0.723958,0.019028,7560000


In [33]:
# Utilizando a livaria 'ta'

# Add bollinger band high indicator filling NaN values
output['bb_high_indicator'] = bollinger_hband_indicator(output["Close"], n=20, ndev=2, fillna=True)

# Add bollinger band low indicator filling NaN values
output['bb_low_indicator'] = bollinger_lband_indicator(output["Close"], n=20, ndev=2, fillna=True)

# Adicionar Money Flow Index
output['MoneyFlowIndex'] = money_flow_index(output["High"],output["Low"],output["Close"],output["Volume"],n=14, fillna=True)

# Adicionar AwesomeOscillator
output['AwesomeOscillator'] = ao(output["High"],output["Low"], s=5, l=34, fillna=True)

# Adicionar Stochatic Oscillator
output['StochaticOscillator'] = stoch(output["High"],output["Low"],output["Close"], n=14, fillna=True)

# Adicionar Stochatic Oscillator Signal
output['StochaticOscillatorSignal'] = stoch_signal(output["High"],output["Low"],output["Close"], n=14, d_n=3, fillna=True)

# Adicionar True Strength Index
output['TrueStrengthIndex'] = tsi(output["Close"], r=25, s=13, fillna=True)

# Adicionar Ultimate Oscillator
output['UltimateOscillator'] = uo(output["High"],output["Low"],output["Close"], s=7, m=14, l=28, ws=4.0, wm=2.0, wl=1.0, fillna=True)

# Adicionar William % R
output['William%R'] = wr(output["High"],output["Low"],output["Close"], lbp=14, fillna=True)

In [34]:
#Calcular o valor RSI 
def RSI(series, period):
 delta = series.diff().dropna()
 u = delta * 0
 d = u.copy()
 u[delta > 0] = delta[delta > 0]
 d[delta < 0] = -delta[delta < 0]
 u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
 u = u.drop(u.index[:(period-1)])
 d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
 d = d.drop(d.index[:(period-1)])
 rs = pd.stats.moments.ewma(u, com=period-1, adjust=False) / \
 pd.stats.moments.ewma(d, com=period-1, adjust=False)
 return 100 - 100 / (1 + rs)

In [35]:
# Adicionar uma coluna(feature) para o valor de RSI
output['RSI'] = RSI(output['Close'], 14)

C:\Users\ruialberto\Anaconda3_v2\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(com=13,min_periods=0,adjust=False,ignore_na=False).mean()
  if sys.path[0] == '':


In [36]:
#Eliminar NaN no campo rsi e obv
rsi_media = output['RSI'].mean()
output['RSI'].fillna(rsi_media, inplace=True)

output.head()

,Date,Open,High,Low,Close,Adj Close,Volume,bb_high_indicator,bb_low_indicator,MoneyFlowIndex,AwesomeOscillator,StochaticOscillator,StochaticOscillatorSignal,TrueStrengthIndex,UltimateOscillator,William%R,RSI
0,1980-01-02,0.718750,0.721354,0.703125,0.703125,0.018480,3451200,0.0,0.0,50.0,0.0,50.0,50.0,0.000000,50.0,-50.0,52.080309
1,1980-01-03,0.703125,0.723958,0.697917,0.721354,0.018959,3960000,0.0,0.0,50.0,0.0,50.0,50.0,100.000000,50.0,-50.0,52.080309
2,1980-01-04,0.721354,0.729167,0.718750,0.729167,0.019165,1694400,0.0,0.0,50.0,0.0,50.0,50.0,100.000000,50.0,-50.0,52.080309
3,1980-01-07,0.729167,0.731771,0.723958,0.726563,0.019096,4396800,0.0,0.0,50.0,0.0,50.0,50.0,95.130679,50.0,-50.0,52.080309
4,1980-01-08,0.726563,0.734375,0.726563,0.731771,0.019233,3244800,0.0,0.0,50.0,0.0,50.0,50.0,92.962798,50.0,-50.0,52.080309


In [37]:
# Adicionar features (MM4,MM9,MM18) e uma label (1=Subiu, 0=Desceu)

output['Change'] = output['Close'].pct_change()
#UP se percentagem 'maior ou igual a 0,6%'

#output['UP'] = np.where(output['Change'] < -0.005 ,1.0,0.0) 

output['UP'] = np.where(output['Change'] >= 0.006,1,0)
output['MM4'] = output['Close'].rolling(window=4).mean()
output['MM9'] = output['Close'].rolling(window=9).mean()
output['MM18'] = output['Close'].rolling(window=18).mean()
output['CrossMM'] = np.where(output['MM4'] > output['MM18'],1.0,0.0)
output['CrossPrice'] = np.where(output['Open'] > output['MM18'],1.0,0.0)

In [38]:
output.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,bb_high_indicator,bb_low_indicator,MoneyFlowIndex,...,UltimateOscillator,William%R,RSI,Change,UP,MM4,MM9,MM18,CrossMM,CrossPrice
0,1980-01-02,0.718750,0.721354,0.703125,0.703125,0.018480,3451200,0.0,0.0,50.0,...,50.0,-50.0,52.080309,NaN,0,NaN,NaN,NaN,0.0,0.0
1,1980-01-03,0.703125,0.723958,0.697917,0.721354,0.018959,3960000,0.0,0.0,50.0,...,50.0,-50.0,52.080309,0.025926,1,NaN,NaN,NaN,0.0,0.0
2,1980-01-04,0.721354,0.729167,0.718750,0.729167,0.019165,1694400,0.0,0.0,50.0,...,50.0,-50.0,52.080309,0.010831,1,NaN,NaN,NaN,0.0,0.0
3,1980-01-07,0.729167,0.731771,0.723958,0.726563,0.019096,4396800,0.0,0.0,50.0,...,50.0,-50.0,52.080309,-0.003571,0,0.720052,NaN,NaN,0.0,0.0
4,1980-01-08,0.726563,0.734375,0.726563,0.731771,0.019233,3244800,0.0,0.0,50.0,...,50.0,-50.0,52.080309,0.007168,1,0.727214,NaN,NaN,0.0,0.0


In [39]:
# Preencher os dados em falta

media4 = output['MM4'].mean()
media9 = output['MM9'].mean()
media18 = output['MM18'].mean()

output['MM4'].fillna(media4, inplace=True)
output['MM9'].fillna(media9, inplace=True)
output['MM18'].fillna(media18, inplace=True)
output['Change'].fillna(0, inplace=True)

In [40]:
output.head(15)

,Date,Open,High,Low,Close,Adj Close,Volume,bb_high_indicator,bb_low_indicator,MoneyFlowIndex,...,UltimateOscillator,William%R,RSI,Change,UP,MM4,MM9,MM18,CrossMM,CrossPrice
0,1980-01-02,0.718750,0.721354,0.703125,0.703125,0.018480,3451200,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,0.000000,0,36.396759,36.397601,36.399115,0.0,0.0
1,1980-01-03,0.703125,0.723958,0.697917,0.721354,0.018959,3960000,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,0.025926,1,36.396759,36.397601,36.399115,0.0,0.0
2,1980-01-04,0.721354,0.729167,0.718750,0.729167,0.019165,1694400,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,0.010831,1,36.396759,36.397601,36.399115,0.0,0.0
3,1980-01-07,0.729167,0.731771,0.723958,0.726563,0.019096,4396800,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,-0.003571,0,0.720052,36.397601,36.399115,0.0,0.0
4,1980-01-08,0.726563,0.734375,0.726563,0.731771,0.019233,3244800,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,0.007168,1,0.727214,36.397601,36.399115,0.0,0.0
5,1980-01-09,0.731771,0.744792,0.729167,0.736979,0.019370,11568000,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,0.007117,1,0.731120,36.397601,36.399115,0.0,0.0
6,1980-01-10,0.736979,0.744792,0.731771,0.736979,0.019370,19444800,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,0.000000,0,0.733073,36.397601,36.399115,0.0,0.0
7,1980-01-11,0.736979,0.750000,0.734375,0.739583,0.019438,6240000,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,0.003533,0,0.736328,36.397601,36.399115,0.0,0.0
8,1980-01-14,0.739583,0.744792,0.729167,0.736979,0.019370,2145600,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,-0.003521,0,0.737630,0.729167,36.399115,0.0,0.0
9,1980-01-15,0.736979,0.736979,0.718750,0.723958,0.019028,7560000,0.0,0.0,50.000000,...,50.0,-50.000000,52.080309,-0.017668,0,0.734375,0.731481,36.399115,0.0,0.0


In [41]:
output.tail(5)

,Date,Open,High,Low,Close,Adj Close,Volume,bb_high_indicator,bb_low_indicator,MoneyFlowIndex,...,UltimateOscillator,William%R,RSI,Change,UP,MM4,MM9,MM18,CrossMM,CrossPrice
89599,2019-02-11,42.279999,42.310001,41.509998,41.680000,41.680000,25713600,0.0,0.0,47.085568,...,52.738289,-39.452049,46.644384,-0.013024,0,41.985001,42.155556,41.823889,1.0,1.0
89600,2019-02-12,41.709999,42.009998,41.389999,41.869999,41.869999,28975500,0.0,0.0,53.560604,...,53.410363,-34.246594,47.997324,0.004559,0,41.870000,42.187778,41.810556,1.0,0.0
89601,2019-02-13,41.919998,42.060001,41.389999,41.700001,41.700001,23488800,0.0,0.0,54.727806,...,54.405187,-38.904076,46.852578,-0.004060,0,41.870000,42.104445,41.767778,1.0,1.0
89602,2019-02-14,41.700001,42.119999,41.619999,41.970001,41.970001,18847200,0.0,0.0,61.706190,...,59.565226,-31.506812,48.935697,0.006475,1,41.805000,42.003334,41.736667,1.0,0.0
89603,2019-02-15,42.380001,42.509998,42.080002,42.400002,42.400002,18186900,0.0,0.0,69.632965,...,60.568364,-19.725956,52.152202,0.010245,1,41.985001,41.998890,41.743889,1.0,1.0


In [42]:
output.keys()

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'bb_high_indicator', 'bb_low_indicator', 'MoneyFlowIndex',
       'AwesomeOscillator', 'StochaticOscillator', 'StochaticOscillatorSignal',
       'TrueStrengthIndex', 'UltimateOscillator', 'William%R', 'RSI', 'Change',
       'UP', 'MM4', 'MM9', 'MM18', 'CrossMM', 'CrossPrice'],
      dtype='object')

In [43]:
#output.to_csv('dow_jones_-005.csv')

In [44]:
corr_matrix = output.corr()
corr_matrix["UP"].sort_values(ascending=False)

UP                           1.000000
Change                       0.610691
William%R                    0.329633
StochaticOscillator          0.329633
bb_high_indicator            0.257597
RSI                          0.222557
UltimateOscillator           0.192628
MoneyFlowIndex               0.153680
StochaticOscillatorSignal    0.084062
Volume                       0.062565
CrossMM                      0.023569
CrossPrice                   0.004788
AwesomeOscillator           -0.018456
TrueStrengthIndex           -0.019265
Close                       -0.025149
Adj Close                   -0.027270
High                        -0.028254
Low                         -0.030254
MM4                         -0.032772
MM18                        -0.032818
MM9                         -0.033246
Open                        -0.033288
bb_low_indicator            -0.138822
Name: UP, dtype: float64

In [45]:
# Split train/sets
from sklearn.model_selection import train_test_split

# Criar tabela com 'features', e com respectiva label (UP)
y = output['UP']
#X = output[['RSI','Volume','MoneyFlowIndex','bb_high_indicator','CrossMM']]
X = output[['Volume','High','Low','RSI','CrossMM','CrossPrice','bb_low_indicator','bb_high_indicator','MoneyFlowIndex','Open','MM4','MM9','MM18','StochaticOscillator', 'William%R', 'UltimateOscillator', 'StochaticOscillatorSignal' ]]

#X = output.drop(['UP'],axis=1)
#Split 
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [46]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(67203, 17)
(22401, 17)
(67203,)
(22401,)


In [47]:
X_test.tail()


,Volume,High,Low,RSI,CrossMM,CrossPrice,bb_low_indicator,bb_high_indicator,MoneyFlowIndex,Open,MM4,MM9,MM18,StochaticOscillator,William%R,UltimateOscillator,StochaticOscillatorSignal
49716,4991800,36.187500,35.093750,45.262350,0.0,0.0,0.0,0.0,47.917077,36.000000,35.898437,35.944444,37.022569,18.919340,-81.080660,49.328109,28.501303
68493,9045000,36.799999,36.310001,47.676598,0.0,0.0,0.0,0.0,52.295752,36.450001,36.800001,37.143333,37.361111,18.224299,-81.775701,46.884252,11.785514
82992,7225200,6.447917,6.333333,53.566889,1.0,1.0,0.0,0.0,57.755393,6.427083,6.351563,6.268519,6.249421,58.695611,-41.304389,57.306056,77.626665
84144,12261600,17.854166,17.125000,73.362599,1.0,1.0,0.0,1.0,55.270836,17.145834,17.244792,17.115741,16.946759,98.529505,-1.470495,63.632821,81.623694
72044,10136700,80.820000,80.300003,48.411421,0.0,0.0,0.0,0.0,54.987681,80.339996,80.324999,80.248889,80.518889,68.944109,-31.055891,52.858353,66.770197


In [48]:
y_test.tail()

49716    0
68493    0
82992    0
84144    1
72044    0
Name: UP, dtype: int32

In [49]:
#Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_sca = sc.fit_transform(X_train)
X_test_sca = sc.transform(X_test)

In [50]:
X_train_sca

array([[-0.10384951,  0.24958081,  0.25817738, ...,  1.39254967,
         0.50064449,  1.12285875],
       [ 2.81028078, -0.86461125, -0.86293014, ..., -0.24554092,
         0.17460128, -0.15334186],
       [ 0.11720184,  0.54613202,  0.53922898, ...,  0.66766687,
         0.42792635,  0.1527303 ],
       ..., 
       [ 1.07235445, -0.17580815, -0.17686812, ...,  0.7375069 ,
         2.49210841,  1.20123264],
       [-0.00681925, -0.88501038, -0.88375347, ..., -1.72792002,
        -1.73165872, -1.50926052],
       [-0.57031737, -0.85197804, -0.85004789, ...,  0.45045746,
         0.12945599,  0.64462199]])

In [51]:
### Escolher o modelo (Logistic Regression), fit model(treinar o modelo), efectuar previsões, e medir resultados)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#Escolher modelo
log_reg = LogisticRegression(C=100)
#Treinar o modelo(fit model)
log_reg.fit(X_train_sca,y_train)
#Efectuar previsões
y_pred = log_reg.predict(X_test_sca)
#Medir resultados previstos com os dados reais

print('Accuracy Score:', accuracy_score(y_test,y_pred))
#print('X_test_sca:', X_test_sca[-1])
print('Valores Reais: \n', y_test[17350:17360])
print('Valores Calculados: ', y_pred[17350:17360])

Accuracy Score: 0.864648899603
Valores Reais: 
 61520    1
330      1
88273    1
83406    1
30165    1
15943    1
23338    0
59686    0
34180    1
84155    0
Name: UP, dtype: int32
Valores Calculados:  [1 1 1 0 1 1 0 1 1 0]


In [52]:
# Confusion matrix

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

# Efectuar uma previsão
y_train_pred_log = cross_val_predict(log_reg, X_train_sca, y_train, cv=3)

# calcular confusion matrix entre a previsão anterior e os resultados reais(y_train)
confusion_matrix(y_train,y_train_pred_log)

array([[41119,  3680],
       [ 5385, 17019]])

In [53]:
# F1_score - (precision and recall)

from sklearn.metrics import precision_score, recall_score, f1_score

print('Performance de acertos: ',precision_score(y_train, y_train_pred_log))
print('True Positive Rate: ',recall_score(y_train,y_train_pred_log))
print('F1 Score: ',f1_score(y_train, y_train_pred_log))

Performance de acertos:  0.822213633509
True Positive Rate:  0.759641135512
F1 Score:  0.789689812774


In [54]:
import pickle

In [55]:
with open('meu_modelo','wb') as f:
    pickle.dump(log_reg,f)